# Sherlock Holmes Stories Markov Chain Model NLP

## Introduction

The purpose of this model is to generate text that is similar to `Shelock Holmes Stories` using `Markov Chains` approach. To build a Markov Chain we need states that will be words of the text and transitions that are the probabilities of going among the states. To find the transition probabilities we traverse the text and find the conditional probability of each word following another. So basically, the probability that the next word (state) will be `j` given that the current word (state) is `i` based on the adjacency list. We can cosider the transitions as statistical properties of the text data and because of this the model will produce random text which is similar to the original text. To generate the text, the model will do a walk (traverse by following the transition probabilities) from a innitial optional (given by us) state (word) and go to the next states. Each state (word) we visited will be part of our text. This can be considered a directed graph with weights traversal, that each node is a state (word) and the edges will define the transitions (probability to be adjacency).

## Implementation

### Installing required Python modules

In [ ]:
%pip install tokenizer
%pip install nltk

### Importing required Python modules

In [1]:
import os
import re
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

#### Fixing NLTK

In [ ]:
import nltk
nltk.download('punkt')

### Read Shelock Holmes stories

In [2]:
def read_all_stories(path: str) -> list:
    stories: list = []
    
    for _, _, files in os.walk(path):
        for file in files:
            with open(path + file) as f:
                for line in f:
                    line = line.strip() # remove spaces around the string
                    # Each story in the dataset has the LICENSE
                    # in the end of the file which will be ignored
                    if line == "----------": break 
                    if line != '': stories.append(line)
                        
    return stories

story_path: str =  "./sherlock-stories-data-set/"

stories: list = read_all_stories(story_path)

assert len(stories) > 0

### Clean the data

Clean dataset required for `NLP`.

In [3]:
def clean_data(data: list) -> list:
    cleaned_data: list = []
    for text in data:
        text = text.lower()
        text = re.sub(r"[,.\"\'!@#$%^&*(){}?/;`~:<>+=-\\]", "", text)
        tokens: list = word_tokenize(text)
        words = [word for word in tokens if word.isalpha()]
        cleaned_data += words
    return cleaned_data

cleaned_stories: list = clean_data(stories)

assert len(cleaned_stories) > 0

### Markov Model

The function will accept also the n_gram which will be used to define number of words in a state (sequence of n words). This will give more context so, the text produced by this model will probably be similar and make more sense

In [4]:
def create_markov_model(cleaned_stories: list, n_gram: int = 1) -> dict:
    markov_model: dict = {}
        
    for i in range(len(cleaned_stories) - n_gram - 1):
        
        current_state: str = ""
        next_state: str = ""
        
        for j in range(n_gram):
            current_state += cleaned_stories[i + j] + " "
            next_state += cleaned_stories[i + j + n_gram] + " "
            
        current_state = current_state[:-1]
        next_state = next_state[:-1]
        
        if current_state not in markov_model:
            markov_model[current_state] = {}
            markov_model[current_state][next_state] = 1
        else:
            if next_state in markov_model[current_state]:
                markov_model[current_state][next_state] += 1
            else:
                markov_model[current_state][next_state] = 1
    
    # Calculating transition probabilities
    for current_state, transition in markov_model.items():
        total = sum(transition.values())
        for state, count in transition.items():
            markov_model[current_state][state] = count / total
        
    return markov_model

markov_model: dict = create_markov_model(cleaned_stories, 2)

assert len(markov_model.keys()) > 0

#### Test the model

In [5]:
# All transitions from `dear holmes` state
print(markov_model['dear holmes'])

{'said i': 0.07017543859649122, 'he has': 0.07017543859649122, 'oh yes': 0.07017543859649122, 'i have': 0.07017543859649122, 'i thought': 0.07017543859649122, 'i ejaculated': 0.07017543859649122, 'what do': 0.07017543859649122, 'i exclaimed': 0.07017543859649122, 'am i': 0.05263157894736842, 'my previous': 0.05263157894736842, 'if i': 0.05263157894736842, 'and tell': 0.05263157894736842, 'that i': 0.05263157894736842, 'i fear': 0.07017543859649122, 'you are': 0.05263157894736842, 'it is': 0.05263157894736842}


In [6]:
# All transitions from `that great` state
print(markov_model['that great'])

{'forest which': 0.12903225806451613, 'brain of': 0.12903225806451613, 'cesspool into': 0.0967741935483871, 'emporium proved': 0.12903225806451613, 'grimpen mire': 0.0967741935483871, 'city so': 0.0967741935483871, 'rich corporations': 0.0967741935483871, 'developments are': 0.12903225806451613, 'trunk of': 0.0967741935483871}


### Generate story

In [7]:
def generate_story(markov_model: dict, start: str, length: int = 100) -> str:
    story: str = start + " "
    current_state = start
    next_state = {}
    
    for _ in range(length):
        next_state = random.choices(list(markov_model[current_state].keys()),
                                    list(markov_model[current_state].values()))[0]
        
        current_state = next_state
        
        story += current_state + " "
        
    return story

### Generating some random stories with different innitial states

In [8]:
# Story 1
print(generate_story(markov_model, "sherlock holmes"), 300)

sherlock holmes the mention of my province there said he how do you think he is frightened for his own but before putting it on he rushed across the glade come here as i have explained are the historian of this bunch well dr watson youve never had any family we live in a mining camp and wandered through woods or mountains so that we had gained a good two hundred yards of the when i was in the house instantly why did she really want surely her own statement is clear enough against you and not for some cause that was the view across the sky with half a million sterling at the window each time that i was mad insane you have done me the honour sir to see sir henry ah here he is sir henry baskerville had consulted me spotted who i was well bribed to help him with the green patches of rushes nothing stirred over the eye and read for yourself he cried ill be perfectly frank with us the secret the more so it is perceptible were it mixed with any idea of murder there were no other footsteps sav

In [9]:
# Story 2
print(generate_story(markov_model, "the case"), 500)

the case you shall have your opinion though i repeatedly told her mother when she reached the spot at the time a deduction which i advocated in that cornish air well filled with spring flowers towards this garden door is usually kept in a broken armchair in the world who knew where is bannister now he shrugged his shoulders i have hitherto confined my investigations to this world said he and his face flushed with anger and he shook his head granting that the occasion of my marriage and my wife or how could you know nothing you are yourself i presume yes when the alarm was given half an hour of trouble our hands instinctively sought for each other within the last hours become much clearer and in some way and if you can muzzle that one of you mcmurdo he had nerves of steel she responded beautifully the spring works they fasten in an honoured old age the sign of four and feet without the smallest problems are of interest to the injured man and turned it over to anyone he is gone he said b

In [10]:
# Story 3
print(generate_story(markov_model, "dear holmes"), 180)

dear holmes and tell you more than a pair of very small dark fellow with his honour cleared and that his mood was particularly anxious to watch him from over yonder i think it must be limited i adopted the expedient of cutting out the words is numbered in the latter case you would ask me whom i did not desire you to start with without the further end of a successful mining claim at a loss to know sir sinking her voice into an impressive whisper i am afraid rance that you will start at once for all off and why should he leave the room in which they did on tiptoe the elder man first the maid thought that young smith was stretched upon the floor and yet i could but silence that foul tongue i did it do you want here the man lived and i set forth to find a dog at the corner of pauls wharf which could tell my terrible story so far as i have seen this old man stirred god bless you sir and all the proofs which i could not help the palate the safe next morning he took a paper of sandwiches  180